#### Загрузка из Json

In [6]:
import pandas as pd
import numpy as np
import json
import glob
import codecs

In [7]:
# Путь
path = 'Data'
file = glob.glob(path + "/*.json")

In [8]:
df_data=[]
for filename in file:
    name = filename.split("\\")[-1][:-5]
    
    with codecs.open(filename, 'r', 'utf-8-sig') as json_file:  
        data = json.load(json_file)
           
    for article in data['refs']:
        if article!=None:
            df_data.append([name, article[0], article[1]['day'], article[1]['month'], article[1]['time']])

df_full=pd.DataFrame(data=df_data,columns=['Название файла', 'Пост', 'день публикации', 'месяц публикации', 'время публикации'])

In [17]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Название файла    1112 non-null   object
 1   Пост              1112 non-null   object
 2   день публикации   1112 non-null   object
 3   месяц публикации  1112 non-null   object
 4   время публикации  1112 non-null   object
dtypes: object(5)
memory usage: 43.6+ KB


In [57]:
df_full = df_full.drop(df_full.index[0:790])

In [58]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 1011 to 1111
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Название файла    100 non-null    object
 1   Пост              100 non-null    object
 2   день публикации   100 non-null    object
 3   месяц публикации  100 non-null    object
 4   время публикации  100 non-null    object
dtypes: object(5)
memory usage: 4.7+ KB


#### Обработка текста

In [59]:
import pymorphy2
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [60]:
sw = stopwords.words('russian') # стоп слова
morph = pymorphy2.MorphAnalyzer() # для нормализации

In [61]:
def clear_text(text):
    text = text.lower() # приводим все в нижний реестр
    text = re.sub(r'[^а-яё ]','', str(text)) # удаление пунктуации
    tokens = word_tokenize(text, language="russian") # токенизация
    tokens = [morph.parse(i)[0].normal_form for i in tokens] # нормализация
    tokens = [ i for i in tokens if i not in sw and len(i) > 3] # не включаем стоп-слова
    return tokens

In [62]:
df_full['lemmatize_tokens'] = df_full['Пост'].apply(clear_text)

In [67]:
df_full['clear_text'] = df_full['lemmatize_tokens'].apply(lambda x: " ".join(x))

In [71]:
df_full.to_csv('data2.csv', index=False)

#### Векторизация и поиск ngram

In [72]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer